In [4]:
#Supervised Learning -> Classifier

In [153]:
# lIBRARY
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import tensorflow as tf
from keras.models import Model, Sequential
from keras.layers import Input, Activation, Dense
from keras.optimizers import SGD
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split

In [154]:
# Import Data Source
df = pd.read_excel('kecelakaanlalin.xlsx')

In [155]:
# Cleaning
def cleaning():
    # drop nan value
    if df.isna().sum().sum() > 0:
        df.replace(0, np.nan, inplace=True)
        df.dropna(inplace=True)

    if df.duplicated().sum() > 0:
        df.drop_duplicates(inplace=True)
cleaning()

In [156]:
# Feature Selection
def tingkatkeparahan(cols):
    if cols == 'Luka Ringan':
        return 0
    elif cols == 'Meninggal Dunia':
        return 1
    elif cols == 'Luka Berat':
        return 3

#change into numeric categorical variable
df['result'] = df['TingkatKeparahan'].apply(tingkatkeparahan)


#change all into dummies variable for machine learning
df_dummies = pd.get_dummies(df, columns=['PekerjaanLawan','PekerjaanKorban','JenisKelaminLawan','JenisKelaminKorban',
                           'JenisKecelakaan','UsiaLawan','UsiaKorban','PeranKorbanDalamKecelakaan','JenisKendaraanKorban',
                           'JenisKendaraanLawan','WaktuKecelakaan'])

#drop column doesn need for prediction
df_dummies.drop(['No','Hari/tgl Kejadian','TempatKejadian','TingkatKeparahan'],axis=1)

# feature more than 0.1
corr_matrix = df_dummies.corr()
corr_column = corr_matrix[(corr_matrix['result']>0.1)]['result']






In [157]:
# Testing

In [158]:
corr_column

result                                                     1.000000
PekerjaanLawan_Dokter                                      0.151690
PekerjaanLawan_Karyawan BUMN                               0.151690
PekerjaanKorban_Buruh                                      0.131751
JenisKecelakaan_T-M                                        0.168535
PeranKorbanDalamKecelakaan_Pengguna Jalan Non Kendaraan    0.164393
JenisKendaraanLawan_Kendaraan  Roda Empat                  0.161751
JenisKendaraanLawan_Kendaraan Roda Empat                   0.113187
Name: result, dtype: float64

In [159]:
df_column = df_dummies[corr_column.index]
df_column.iloc[:,:-2]

,result,PekerjaanLawan_Dokter,PekerjaanLawan_Karyawan BUMN,PekerjaanKorban_Buruh,JenisKecelakaan_T-M,PeranKorbanDalamKecelakaan_Pengguna Jalan Non Kendaraan
0,0,0,0,0,0,0
2,0,0,0,0,1,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
6,0,0,0,0,0,0
...,...,...,...,...,...,...
415,0,0,0,0,0,0
416,0,0,0,0,0,0
417,0,0,0,0,0,0
420,0,0,0,0,0,0


In [160]:
X = df_column.loc[:, df_column.columns != 'result']

y = df_column.iloc[:,:1]

In [161]:
df_y = pd.get_dummies(df, columns=['TingkatKeparahan'])
df_y = df_y.iloc[:,-3:]

In [162]:
df_y

,TingkatKeparahan_Luka Berat,TingkatKeparahan_Luka Ringan,TingkatKeparahan_Meninggal Dunia
0,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0
6,0,1,0
...,...,...,...
415,0,1,0
416,0,1,0
417,0,1,0
420,0,1,0


In [163]:
X_train, X_test, y_train , y_test = train_test_split(X,df_y,test_size=0.3,random_state=101)

In [168]:
inputs = Input(shape=(7,))
h_layer = Dense(7, activation='sigmoid')(inputs)

# Softmax Activation for Multiclass Classification
outputs = Dense(3, activation='softmax')(h_layer)

model = Model(inputs=inputs, outputs=outputs)

# Optimizer / Update Rule
sgd = SGD(lr=0.001)

# Compile the model with Cross Entropy Loss
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])


In [169]:
model.fit(X_train, y_train, batch_size=7, epochs=50, verbose=1, validation_data=(X_test, y_test))
model.save_weights('weights.h5')

# Predict all Validation data
predict = model.predict(X_test)

# Visualize Prediction
df_pa = pd.DataFrame(predict)
df_pa.columns = [ 'TingkatKeparahan_Luka Berat', 'TingkatKeparahan_Luka Ringan', 'TingkatKeparahan_Meninggal Dunia']
# df_pa.index = val_data[:,0]
print(df_pa)

Train on 260 samples, validate on 112 samples
Epoch 1/50
260/260 [==============================] - 0s 421us/step - loss: 1.0645 - accuracy: 0.1615 - val_loss: 1.0441 - val_accuracy: 0.1429
Epoch 2/50
260/260 [==============================] - 0s 162us/step - loss: 1.0261 - accuracy: 0.1692 - val_loss: 1.0059 - val_accuracy: 0.1429
Epoch 3/50
260/260 [==============================] - 0s 162us/step - loss: 0.9905 - accuracy: 0.5000 - val_loss: 0.9704 - val_accuracy: 0.8571
Epoch 4/50
260/260 [==============================] - 0s 160us/step - loss: 0.9574 - accuracy: 0.8385 - val_loss: 0.9375 - val_accuracy: 0.8571
Epoch 5/50
260/260 [==============================] - 0s 146us/step - loss: 0.9269 - accuracy: 0.8385 - val_loss: 0.9071 - val_accuracy: 0.8571
Epoch 6/50
260/260 [==============================] - 0s 148us/step - loss: 0.8987 - accuracy: 0.8385 - val_loss: 0.8789 - val_accuracy: 0.8571
Epoch 7/50
260/260 [==============================] - 0s 163us/step - loss: 0.8726 - accur

In [173]:
y_test.head()

,TingkatKeparahan_Luka Berat,TingkatKeparahan_Luka Ringan,TingkatKeparahan_Meninggal Dunia
112,0,1,0
47,0,1,0
104,0,1,0
386,0,1,0
192,0,0,1


In [174]:
df_pa.head()

,TingkatKeparahan_Luka Berat,TingkatKeparahan_Luka Ringan,TingkatKeparahan_Meninggal Dunia
0,0.086019,0.787344,0.126637
1,0.086019,0.787344,0.126637
2,0.087914,0.769558,0.142528
3,0.096116,0.750814,0.153070
4,0.096116,0.750814,0.153070


In [172]:
test_loss, test_acc = Sequential().evaluate(df_pa, y_test)
print('test_acc:', test_acc, 'test_loss', test_loss)

ValueError: Please provide as model inputs either a single array or a list of arrays. You passed: x=     TingkatKeparahan_Luka Berat  TingkatKeparahan_Luka Ringan  \
0                       0.086019                      0.787344   
1                       0.086019                      0.787344   
2                       0.087914                      0.769558   
3                       0.096116                      0.750814   
4                       0.096116                      0.750814   
..                           ...                           ...   
107                     0.096116                      0.750814   
108                     0.096116                      0.750814   
109                     0.096116                      0.750814   
110                     0.096116                      0.750814   
111                     0.096116                      0.750814   

     TingkatKeparahan_Meninggal Dunia  
0                            0.126637  
1                          

In [85]:
model = Sequential()

# Inputing the first layer with input dimensions
model.add(Dense(5, 
                activation='relu',  
                input_dim=7,
                kernel_initializer='uniform'))
#The argument being passed to each Dense layer (18) is the number of hidden units of the layer. 
# A hidden unit is a dimension in the representation space of the layer.

#Stacks of Dense layers with relu activations can solve a wide range of problems
#(including sentiment classification), and you’ll likely use them frequently.

# Adding an Dropout layer to previne from overfitting
model.add(Dropout(0.50))

#adding second hidden layer 
model.add(Dense(60,
                kernel_initializer='uniform',
                activation='relu'))

# Adding another Dropout layer
model.add(Dropout(0.50))

model.add(Dense(1,
                kernel_initializer='uniform',
                activation='sigmoid'))
#With such a scalar sigmoid output on a binary classification problem, the loss
#function you should use is binary_crossentropy

#Visualizing the model
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 5)                 40        
_________________________________________________________________
dropout_7 (Dropout)          (None, 5)                 0         
_________________________________________________________________
dense_23 (Dense)             (None, 60)                360       
_________________________________________________________________
dropout_8 (Dropout)          (None, 60)                0         
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 61        
Total params: 461
Trainable params: 461
Non-trainable params: 0
_________________________________________________________________


In [66]:
#Creating an Stochastic Gradient Descent
sgd = SGD(lr = 0.01, momentum = 0.9)

# Compiling our model
model.compile(optimizer = sgd, 
                   loss = 'binary_crossentropy', 
                   metrics = ['accuracy'])
#optimizers list
#optimizers['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']

# Fitting the ANN to the Training set
model.fit(X_train, y_train, 
               batch_size = 60, 
               epochs = 30, verbose=2)

Epoch 1/30
 - 0s - loss: 0.6912 - accuracy: 0.8423
Epoch 2/30
 - 0s - loss: 0.6782 - accuracy: 0.8423
Epoch 3/30
 - 0s - loss: 0.6588 - accuracy: 0.8423
Epoch 4/30
 - 0s - loss: 0.6366 - accuracy: 0.8423
Epoch 5/30
 - 0s - loss: 0.6182 - accuracy: 0.8423
Epoch 6/30
 - 0s - loss: 0.5999 - accuracy: 0.8423
Epoch 7/30
 - 0s - loss: 0.5864 - accuracy: 0.8423
Epoch 8/30
 - 0s - loss: 0.5761 - accuracy: 0.8423
Epoch 9/30
 - 0s - loss: 0.5664 - accuracy: 0.8423
Epoch 10/30
 - 0s - loss: 0.5613 - accuracy: 0.8423
Epoch 11/30
 - 0s - loss: 0.5560 - accuracy: 0.8423
Epoch 12/30
 - 0s - loss: 0.5543 - accuracy: 0.8423
Epoch 13/30
 - 0s - loss: 0.5503 - accuracy: 0.8423
Epoch 14/30
 - 0s - loss: 0.5490 - accuracy: 0.8423
Epoch 15/30
 - 0s - loss: 0.5482 - accuracy: 0.8423
Epoch 16/30
 - 0s - loss: 0.5475 - accuracy: 0.8423
Epoch 17/30
 - 0s - loss: 0.5469 - accuracy: 0.8423
Epoch 18/30
 - 0s - loss: 0.5462 - accuracy: 0.8423
Epoch 19/30
 - 0s - loss: 0.5458 - accuracy: 0.8423
Epoch 20/30
 - 0s - l

In [67]:
y_preds = model.predict(X_test)


In [68]:
y_preds

array([[0.24216327],
       [0.24216327],
       [0.24215662],
       [0.24206355],
       [0.24206355],
       [0.24206355],
       [0.24206355],
       [0.24206355],
       [0.24215662],
       [0.24206355],
       [0.24206355],
       [0.24206355],
       [0.24206355],
       [0.24206355],
       [0.24206355],
       [0.24206355],
       [0.24206355],
       [0.24206355],
       [0.24206355],
       [0.24206355],
       [0.24206355],
       [0.24206355],
       [0.24215662],
       [0.24216327],
       [0.24215662],
       [0.24216327],
       [0.24206355],
       [0.24206355],
       [0.24205342],
       [0.24206355],
       [0.24206355],
       [0.24206355],
       [0.24206355],
       [0.24215662],
       [0.24215662],
       [0.24206355],
       [0.24206355],
       [0.24206355],
       [0.24206355],
       [0.24206355],
       [0.24206355],
       [0.24206355],
       [0.24206355],
       [0.24207672],
       [0.24206355],
       [0.24216327],
       [0.24206355],
       [0.242

In [50]:
scores = model.evaluate(X_train, y_train, batch_size=30)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100)

SyntaxError: unexpected EOF while parsing (<ipython-input-50-1183a8a8b4b2>, line 2)

In [62]:
df.TingkatKeparahan.unique()

array(['Luka Ringan', 'Meninggal Dunia', 'Luka Berat'], dtype=object)

In [44]:
def tingkatkeparahan(cols):
    if cols == 'Luka Ringan':
        return 0
    elif cols == 'Meninggal Dunia':
        return 1
    elif cols == 'Luka Berat':
        return 3

In [47]:
df['result'] = df['TingkatKeparahan'].apply(tingkatkeparahan)


In [ ]:
df_dummies = pd.get_dummies(df, columns=['PekerjaanLawan','PekerjaanKorban','JenisKelaminLawan','JenisKelaminKorban',
                           'JenisKecelakaan','UsiaLawan','UsiaKorban','PeranKorbanDalamKecelakaan','JenisKendaraanKorban',
                           'JenisKendaraanLawan','WaktuKecelakaan'])

In [ ]:
df_dummies.drop(['No','Hari/tgl Kejadian','TempatKejadian','TingkatKeparahan'],axis=1)

In [66]:
#Selection
corr_matrix = df_dummies.corr()
corr_matrix[(corr_matrix['result']>0.1)]['result']


,No,Hari/tgl Kejadian,TempatKejadian,PekerjaanLawan,PekerjaanKorban,JenisKelaminLawan,JenisKelaminKorban,JenisKecelakaan,UsiaLawan,UsiaKorban,PeranKorbanDalamKecelakaan,JenisKendaraanKorban,JenisKendaraanLawan,WaktuKecelakaan,TingkatKeparahan,result
94,LP/95/I/2018/LL,Sabtu/27 Januari 2018,Jl. Manding-Imogiri,Swasta,Petani,Perempuan,Laki-Laki,D-S,Dewasa,Lansia,Pengguna Kendaraan,Sepeda Motor,Sepeda Motor,Padat Kendaraan,Luka Berat,3
102,LP/103/I/2018/LL,Senin/29 Januari 2018,Jl. Angkruksari,Mahasiswa,IRT,Laki-Laki,Perempuan,T-M,Remaja,Manula,Pengguna Jalan Non Kendaraan,Lain-lain,Sepeda Motor,Sepi Kendaraan,Luka Berat,3
173,LP/174/II/2018/LL,Minggu/25 Februari 2018,Jl. Imogiri Timur,Swasta,Swasta,Laki-Laki,Perempuan,D-S,Dewasa,Dewasa,Pengguna Kendaraan,Sepeda Motor,Sepeda Motor,Padat Kendaraan,Luka Berat,3
182,LP/182/II/2018/LL,Selasa/27 Februari 2018,Jl. Raya Jogja,Mahasiswa,Buruh,Perempuan,Laki-Laki,T-M,Remaja,Lansia,Pengguna Jalan Non Kendaraan,Lain-lain,Sepeda Motor,Sepi Kendaraan,Luka Berat,3
191,LP/191/II/2018/LL,Jumat/2 Maret 2018,Jl. Sugiyo Pranoro,Dokter,Pelajar,Laki-Laki,Laki-Laki,D-S,Manula,Remaja,Pengguna Kendaraan,Sepeda Motor,Kendaraan Roda Empat,Padat Kendaraan,Luka Berat,3
200,LP/200/II/2018/LL,Senin/5 Maret 2018,Jl. Bantul,Swasta,Swasta,Laki-Laki,Perempuan,D-B,Dewasa,Dewasa,Pengguna Kendaraan,Sepeda Motor,Sepeda Motor,Padat Kendaraan,Luka Berat,3
213,LP/213/III/2018/LL,Jumat/9 Maret 2018,Jl. Srandakan,Swasta,Pelajar,Laki-Laki,Laki-Laki,D-S,Remaja,Remaja,Pengguna Kendaraan,Sepeda Motor,Sepeda Motor,Sepi Kendaraan,Luka Berat,3
218,LP/218/III/2018/LL,Sabtu/10 Maret 2018,Jl. Parangtritis,Karyawan BUMN,Swasta,Perempuan,Laki-Laki,S-S,Dewasa,Dewasa,Pengguna Kendaraan,Sepeda Motor,Sepeda Motor,Padat Kendaraan,Luka Berat,3
223,LP/223/III/2018/LL,Minggu/11 Maret 2018,Jl. Yogya-Wonosari,Swasta,Swasta,Perempuan,Laki-Laki,D-S,Lansia,Manula,Pengguna Kendaraan,Sepeda Motor,Sepeda Motor,Padat Kendaraan,Luka Berat,3
238,LP/238/III/2018/LL,Rabu/14 Maret 2018,Jl. Imogiri Timur,Swasta,Buruh,Laki-Laki,Laki-Laki,D-D,Dewasa,Dewasa,Pengguna Kendaraan,Sepeda Motor,Kendaraan Roda Empat,Padat Kendaraan,Luka Berat,3


In [50]:
df.tail(100)

,No,Hari/tgl Kejadian,TempatKejadian,PekerjaanLawan,PekerjaanKorban,JenisKelaminLawan,JenisKelaminKorban,JenisKecelakaan,UsiaLawan,UsiaKorban,PeranKorbanDalamKecelakaan,JenisKendaraanKorban,JenisKendaraanLawan,WaktuKecelakaan,TingkatKeparahan,result
309,LP/308/IV/2018/LL,Selasa/3 April 2018,Jl. Imogiri Timur,Pedagang,Mahasiswa,Laki-Laki,Laki-Laki,D-D,Lansia,Remaja,Pengguna Kendaraan,Sepeda Motor,Kendaraan Roda Empat,Sepi Kendaraan,Luka Ringan,0
311,LP/310/IV/2018/LL,Rabu/4 April 2018,Jl. Berberdan,Swasta,Swasta,Laki-Laki,Laki-Laki,D-S,Dewasa,Dewasa,Pengguna Kendaraan,Sepeda Motor,Sepeda Motor,Sepi Kendaraan,Luka Ringan,0
312,LP/311/IV/2018/LL,Rabu/4 April 2018,Jl. Parangtritis,Swasta,Swasta,Laki-Laki,Perempuan,S-S,Dewasa,Manula,Pengguna Kendaraan,Lain-lain,Sepeda Motor,Padat Kendaraan,Luka Ringan,0
313,LP/312/IV/2018/LL,Rabu/4 April 2018,Jl. Pundong-Klegen,Swasta,Polisi,Laki-Laki,Laki-Laki,D-S,Lansia,Lansia,Pengguna Kendaraan,Sepeda Motor,Kendaraan Lebih Empat Roda,Padat Kendaraan,Luka Ringan,0
314,LP/313/IV/2018/LL,Rabu/4 April 2018,Jl. Jogja-Wonosari,Pelajar,Mahasiswa,Laki-Laki,Laki-Laki,D-D,Remaja,Remaja,Pengguna Kendaraan,Sepeda Motor,Sepeda Motor,Padat Kendaraan,Luka Ringan,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,LP/414/V/2018/LL,Rabu/2 Mei 2018,Jl. Parangtritis,Swasta,Pelajar,Perempuan,Laki-Laki,D-S,Lansia,Remaja,Pengguna Kendaraan,Sepeda Motor,Sepeda Motor,Padat Kendaraan,Luka Ringan,0
416,LP/415/V/2018/LL,Kamis/3 Mei 2018,Jl. Jogja-Wonosari,Swasta,Petani,Laki-Laki,Laki-Laki,D-S,Remaja,Manula,Pengguna Kendaraan,Lain-lain,Sepeda Motor,Sepi Kendaraan,Luka Ringan,0
417,LP/416/V/2018/LL,Kamis/3 Mei 2018,Jl. Parangtritis,Swasta,Swasta,Laki-Laki,Perempuan,D-S,Dewasa,Dewasa,Pengguna Kendaraan,Sepeda Motor,Sepeda Motor,Sepi Kendaraan,Luka Ringan,0
420,LP/419/V/2018/LL,Jumat/4 Mei 2018,Jl. Kampung Worotawang,Buruh,Pelajar,Perempuan,Laki-Laki,S-S,Dewasa,Anak-anak,Pengguna Kendaraan,Lain-lain,Sepeda Motor,Padat Kendaraan,Luka Ringan,0


In [ ]:
# Include , Pekerjaan lawan = categorical , Pekerjaan Korban =  categorical , JenisKelaminLawan = categorical,
# JenisKelaminKorban = categorical , JenisKecelakaan = categorical , UsiaLawan = categorical , UsiaKorban = categorical,
# PeranKorbanDalamKecelakaan = categorical , JenisKendaraanKorban = categorical , JenisKendaraanLawan = categorical,
# WaktuKecelakaan = categorical , TingkatKeparahan = categorical 

In [43]:
# exclude No, Hari , Tempat Kejadian , Jenis Kendaraan Lawan agak aneh (ada value yang sama beda di spasi) ,
# WaktuKecelakaan juga ada value sama beda di spasi

In [67]:
df_dummies = pd.get_dummies(df, columns=['PekerjaanLawan','PekerjaanKorban','JenisKelaminLawan','JenisKelaminKorban',
                           'JenisKecelakaan','UsiaLawan','UsiaKorban','PeranKorbanDalamKecelakaan','JenisKendaraanKorban',
                           'JenisKendaraanLawan','WaktuKecelakaan'])

In [69]:
df_dummies.drop(['No','Hari/tgl Kejadian','TempatKejadian','TingkatKeparahan'],axis=1)

,result,PekerjaanLawan_BUMN,PekerjaanLawan_Buruh,PekerjaanLawan_Buruh Tani,PekerjaanLawan_Dokter,PekerjaanLawan_Guru,PekerjaanLawan_IRT,PekerjaanLawan_Karyawan BUMN,PekerjaanLawan_Karyawan PDAM,PekerjaanLawan_Mahasiswa,...,JenisKendaraanLawan_Kendaraan Lebih Roda Empat,JenisKendaraanLawan_Kendaraan Roda Empat,JenisKendaraanLawan_Kendaraan Roda Empat,JenisKendaraanLawan_Lain-lain,JenisKendaraanLawan_Sepeda Motor,JenisKendaraanLawan_Sepeda Motor,WaktuKecelakaan_Padat Kendaraan,WaktuKecelakaan_Padat Kendaraan,WaktuKecelakaan_Sepi Kendaraan,WaktuKecelakaan_Sepi Kendaraan
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
416,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
417,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
420,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0


In [22]:
df.isna().sum()

No                            0
Hari/tgl Kejadian             0
TempatKejadian                0
PekerjaanLawan                3
PekerjaanKorban               0
JenisKelaminLawan             5
JenisKelaminKorban            0
JenisKecelakaan               0
UsiaLawan                     3
UsiaKorban                    0
PeranKorbanDalamKecelakaan    0
JenisKendaraanKorban          0
JenisKendaraanLawan           0
WaktuKecelakaan               0
TingkatKeparahan              0
dtype: int64

In [23]:
df.duplicated().sum()

0

In [24]:
df.replace(0, np.nan, inplace=True)
df.dropna(inplace=True)

In [25]:
df.describe()

,No,Hari/tgl Kejadian,TempatKejadian,PekerjaanLawan,PekerjaanKorban,JenisKelaminLawan,JenisKelaminKorban,JenisKecelakaan,UsiaLawan,UsiaKorban,PeranKorbanDalamKecelakaan,JenisKendaraanKorban,JenisKendaraanLawan,WaktuKecelakaan,TingkatKeparahan
count,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372
unique,369,119,192,24,23,2,4,6,4,5,2,5,9,4,3
top,LP/288/III/2018/LL,Sabtu/6 Januari 2018,Jl. Parangtritis,Swasta,Swasta,Laki-Laki,Laki-Laki,D-S,Dewasa,Lansia,Pengguna Kendaraan,Sepeda Motor,Sepeda Motor,Sepi Kendaraan,Luka Ringan
freq,2,6,32,174,161,299,222,164,150,115,331,260,260,202,315


In [26]:
pd.set_option('display.max_row', None)
df.head(500)

,No,Hari/tgl Kejadian,TempatKejadian,PekerjaanLawan,PekerjaanKorban,JenisKelaminLawan,JenisKelaminKorban,JenisKecelakaan,UsiaLawan,UsiaKorban,PeranKorbanDalamKecelakaan,JenisKendaraanKorban,JenisKendaraanLawan,WaktuKecelakaan,TingkatKeparahan
0,LP/1/I/2018/LL,Senin/1 Januari 2018,Jl. Kampung Sebapang,Pelajar SMP,Swasta,Laki-Laki,Laki-Laki,D-S,Remaja,Lansia,Pengguna Kendaraan,Lain-lain,Sepeda Motor,Padat Kendaraan,Luka Ringan
2,LP/03/I/2018/LL,Senin/1 Januari 2018,Jl Raya Jogja-Wates,Swasta,Swasta,Laki-Laki,Laki-Laki,T-M,Lansia,Remaja,Pengguna Kendaraan,Sepeda Motor,Kendaraan Roda Empat,Sepi Kendaraan,Luka Ringan
3,LP/04/I/2018/LL,Selasa/2 Januari 2018,Jl. Srandakan,Wiraswasta,Swasta,Laki-Laki,Laki-Laki,D-S,Lansia,Remaja,Pengguna Kendaraan,Sepeda Motor,Kendaraan Roda Empat,Sepi Kendaraan,Luka Ringan
4,LP/05/I/2018/LL,Selasa/2 Januari 2018,Jl. Pleret Dusun Priyan Potorono,Swasta,Pelajar,Laki-Laki,Laki-Laki,D-S,Dewasa,Remaja,Pengguna Kendaraan,Sepeda Motor,Sepeda Motor,Padat Kendaraan,Luka Ringan
6,LP/07/I/2018/LL,Selasa/2 Januari 2018,Jl. Srandakan,Pelajar,Pelajar,Laki-Laki,Laki-Laki,D-S,Remaja,Remaja,Pengguna Kendaraan,Sepeda Motor,Sepeda Motor,Padat Kendaraan,Luka Ringan
7,LP/08/I/2018/LL,Selasa/2 Januari 2018,Jl. Imogiri Barat KM 9,Swasta,Swasta,Laki-Laki,Laki-Laki,S-S,Dewasa,Dewasa,Pengguna Kendaraan,Sepeda Motor,Sepeda Motor,Sepi Kendaraan,Luka Ringan
8,LP/09/I/2018/LL,Selasa/2 Januari 2018,Jl. Samas Dusun Kawi Jureng,Swasta,Pelajar,Laki-Laki,Perempuan,D-B,Dewasa,Dewasa,Pengguna Kendaraan,Sepeda Motor,Sepeda Motor,Sepi Kendaraan,Luka Ringan
9,LP/10/I/2018/LL,Selasa/2 Januari 2018,Jl. Yogya-Wonosari Dusun Sandeyan,BUMN,Swasta,Laki-Laki,Laki-Laki,D-B,Dewasa,Remaja,Pengguna Kendaraan,Sepeda Motor,Sepeda Motor,Sepi Kendaraan,Luka Ringan
11,LP/12/I/2018/LL,"Rabu, 3 Januari 2018",Jl. Parangtritis Simpang 4 Druwo,Swasta,Pelajar,Laki-Laki,Perempuan,S-S,Dewasa,Remaja,Pengguna Kendaraan,Sepeda Motor,Kendaraan Roda Empat,Sepi Kendaraan,Luka Ringan
12,LP/13/I/2018/LL,"Rabu, 3 Januari 2018",Jl. Sedayu Payangan,Swasta,Swasta,Laki-Laki,Laki-Laki,D-D,Remaja,Dewasa,Pengguna Kendaraan,Sepeda Motor,Sepeda Motor,Padat Kendaraan,Luka Ringan


In [27]:

null_columns=df.columns[df.isnull().any()]
df[df.isnull().any(axis=1)][null_columns]

""


In [87]:
corr_matrix = df_dummies.corr()
corr_matrix[(corr_matrix['result']>0.1)]['result']

result                                                     1.000000
PekerjaanLawan_Dokter                                      0.151690
PekerjaanLawan_Karyawan BUMN                               0.151690
PekerjaanKorban_Buruh                                      0.131751
JenisKecelakaan_T-M                                        0.168535
PeranKorbanDalamKecelakaan_Pengguna Jalan Non Kendaraan    0.164393
JenisKendaraanLawan_Kendaraan  Roda Empat                  0.161751
JenisKendaraanLawan_Kendaraan Roda Empat                   0.113187
Name: result, dtype: float64